In [3]:
from collections import defaultdict
from model import IMPALAFruitfly
import torch
import os, copy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import pickle
from ray.air import Checkpoint
from ray.rllib.models import ModelCatalog
import ray
from model import IMPALAFruitfly
import pickle5 as pickle
from wordcloud import WordCloud
from transformers import AutoTokenizer
import json
import gym
from tabulate import tabulate
from ray.rllib.algorithms import impala

In [4]:
#W = torch.load('/pvc/privatefs/afigueroa/fruitfly/models/2022_03_18__133640846698_587934f6/checkpoint_19.pth', map_location='cpu')
cpu_path = '/pvc/privatefs/afigueroa/fruitfly/models/2022_03_18__133640846698_587934f6/checkpoint_19_cpu.pth'
#torch.save(W, cpu_path)

from torch import nn
from transformers import AutoModel, AutoConfig, AutoTokenizer
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.recurrent_net import RecurrentNetwork as TorchRNN
from ray.rllib.utils.annotations import override
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.modelv2 import ModelV2, restore_original_dimensions

from typing import Dict, Any, List

import numpy as np

class IMPALAFruitfly(TorchModelV2, nn.Module):
#class IMPALAFruitfly(TorchRNN, nn.Module):

    def __init__(self, obs_space, action_space, num_outputs, model_config, name, K, k, tokenizer_name="bert-base-uncased", model_path=None):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)

        nn.Module.__init__(self)

      
        config = AutoConfig.from_pretrained(tokenizer_name)
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)#Debugging TODO: REMOVE
        self.sample_count = 0 #Debugging TODO: REMOVE
        self.config = model_config
        self.K = K
        self.k = k
        #self.lstm = torch.nn.LSTM(K, 256, batch_first=True) 
        if model_path is not None:
            self.W = torch.load(model_path)
            #self.W.requires_grad=False
            self.vocab_size = self.W.shape[-1]//2
        else:
            self.vocab_size = config.vocab_size
            self.W = torch.nn.Parameter(torch.randn(K, 2*self.vocab_size, dtype=torch.float32), requires_grad=True)

        self.observation_space = obs_space
        self.action_space = action_space
        self.window_size = 11
        self.e_weight = 0.999 # Weight of energy custom loss
        self.activation = nn.GELU()
        


        self.action_outputs = nn.Linear(K, num_outputs)
        #self.action_outputs = nn.Linear(256, num_outputs)

        self.value_network = nn.Linear(K, 1)
        #self.value_network = nn.Linear(256, 1)
        self.fs = torch.nn.Parameter(torch.ones(self.vocab_size), requires_grad=False) #Acumulated frequencies of tokens
        self._features = None
        #def debug_fn(grads):
        #    return grads*1e6

        #self.encoder.W.register_hook(debug_fn)

    @override(TorchModelV2)
    def forward(self, input_dict: Dict, state: List, seq_lens: Any):
        ids = input_dict["obs"].squeeze(1) 
        
        ids = torch.cat((ids, ids+self.vocab_size), dim=1) # Add the ids to the target part of the sparse vector 
        full_sequence_batch_indices = torch.arange(0, len(ids), dtype=torch.int32).repeat_interleave(input_dict["obs"].shape[-1]).to(input_dict['obs'].device)
        full_sequence_coordinates = torch.stack((full_sequence_batch_indices, input_dict["obs"].reshape(-1))).T
        # Mask out the [PAD] tokens by setting their value to 0, coalesce to shrink the nnzs
        #sparse_values = torch.ones_like(full_sequence_coordinates.T[0])*(input_dict['obs']!=0).reshape(full_sequence_coordinates.T[0].shape) 
        sparse_values = torch.ones_like(full_sequence_coordinates.T[0]).reshape(full_sequence_coordinates.T[0].shape) 
        out_V_A_s = torch.sparse_coo_tensor(full_sequence_coordinates.T, sparse_values, (ids.shape[0], 2*self.vocab_size), dtype=torch.float32).coalesce()

        activations = torch.sparse.mm(out_V_A_s, self.W.T)
        self._features = activations
        
        binary_hash = torch.zeros_like(activations, dtype=torch.bool)
        order = activations.argsort(dim=1, descending=True)
        trues = order[:,:self.k]
        binary_hash = binary_hash.scatter_(dim=1, index = trues, src=torch.ones_like(trues, dtype=torch.bool))

        logits = self.activation(self.action_outputs(binary_hash.float()))
        return  logits, state

    @override(TorchModelV2)
    def custom_loss(self, policy_loss, loss_inputs):
        ids = loss_inputs["obs"].squeeze(1)

        # Updating the frequency of the tokens
        if self.sample_count < 1e9:
          full_sequence_batch_indices = torch.arange(0, len(ids), dtype=torch.int32).repeat_interleave(loss_inputs["obs"].shape[-1]).to(loss_inputs['obs'].device)
          full_sequence_coordinates = torch.stack((full_sequence_batch_indices, loss_inputs["obs"].reshape(-1))).T
          out_V_A_s = torch.sparse_coo_tensor(full_sequence_coordinates.T, torch.ones_like(full_sequence_coordinates.T[0]), (ids.shape[0], 2*self.vocab_size), dtype=torch.float32)
          fs = torch.sparse.sum(out_V_A_s, dim=0)
          self.fs[fs.indices().squeeze()] += fs.values()

        input_ids = ids.unfold(1, self.window_size, 1).clone().reshape(-1, self.window_size)
        input_ids[:,(self.window_size//2)] += self.vocab_size 
        indices_batch = torch.arange(0,len(input_ids), dtype=torch.int32).repeat_interleave(self.window_size).to(input_ids.device)
        coordinates = torch.stack((indices_batch, input_ids.reshape(-1))).T

        Ps = 1/self.fs[coordinates.T[1]%self.vocab_size].reshape(input_ids.shape[0], input_ids.shape[1])
        batch_size = input_ids.shape[0] # Batch size in the sense of the windowed sequences
        # Mask out the values of the [PAD] tokens, coalesce the sparse tensor to reduce the number of mms, coalesce() to reduce the nnz hence the multiplications of repeated tokens
        sparse_values = torch.logical_and(input_ids!=0,input_ids!=self.vocab_size).float().reshape(coordinates.T[0].shape)
        
        V_A_s = torch.sparse_coo_tensor(coordinates.T,torch.ones_like(coordinates.T[0]),(batch_size,2*self.vocab_size),dtype=torch.float32).coalesce()#.to_sparse_csr()
        #breakpoint()
        #Inv_P_s = torch.sparse_coo_tensor(coordinates.T,Ps.reshape(-1),(batch_size,2*self.vocab_size),dtype=torch.float32)#.to_sparse_csr()

        V_AxWT = torch.sparse.mm(V_A_s, self.W.T)
        mu = V_AxWT.argmax(axis=1)
        W_mu_nonzero = torch.gather(self.W[mu], 1, coordinates.T[1].reshape(batch_size,-1))
        alpha = (W_mu_nonzero*Ps).sum(axis=1)
        denominator = torch.linalg.norm(self.W, dim = 1)[mu]
        E = -(alpha/denominator).sum() 
        self.fruit_fly_energy_metric = E.item()
        self.fruit_fly_energy_inverse_metric = 1/(E.item())
        self.policy_loss_metric = np.mean([loss.item() for loss in policy_loss])
        
        self.sample_count +=1
        #if self.sample_count == 10:
        #    breakpoint()
        return [(1 - self.e_weight)*loss_+ self.e_weight*E for loss_ in policy_loss]
        #return [E] + policy_loss

    @override(TorchModelV2)    
    def value_function(self):
        assert self._features is not None, "Must call forward first"
        #return torch.reshape(self._E*self.activation(self.value_network(self._features)), [-1])
        return torch.reshape(self.activation(self.value_network(self._features)), [-1])

    def metrics(self):
        return {
                'policy_loss': self.policy_loss_metric,
                'fruit_fly_energy':self.fruit_fly_energy_metric,
                'fruit_fly_energy_inverse':self.fruit_fly_energy_inverse_metric
                }

In [5]:
def ids2English(tokenizer):
    """
    Closure that returns a function to translate token ids to english given
    a tokenizer.
    """
    full_vocab = copy.deepcopy(tokenizer.vocab)
    #full_vocab.update(tokenizer.added_tokens_encoder.items())
    vocab_inv = { v:k for k,v in full_vocab.items()}
    
    #special_tokens = SPECIAL_TOKENS
    special_tokens = []
    special_token_ids = [full_vocab[t.lower()] for t in special_tokens]
    special_tokens +=['-100 MASK', '[CLS]', '[PAD]', '[MASK]']
    special_token_ids += [-100, full_vocab['[CLS]'], full_vocab['[PAD]'], full_vocab['[MASK]']]

    def parser(input_ids):
        res = []
        i = 0
        while i < len(input_ids):
            t = input_ids[i]
            if t in special_token_ids:
                token_str = special_tokens[special_token_ids.index(t)]
                repetitions = 0
                while i < len(input_ids) and input_ids[i] == t:
                    repetitions += 1
                    i += 1  
                # Trim the []
                token_str = token_str.replace('[','')
                token_str = token_str.replace(']','')
                
                if repetitions == 1:
                    res.append(f"[{token_str}]")
                if repetitions > 1:
                    res.append(f"[{repetitions}X {token_str}]")
            else:
                res.append(vocab_inv[t])
                i += 1
        return " ".join(res)

    return parser

In [6]:
checkpoint_base_url = '/root/ray_results/bert-base-uncased-None/IMPALA_gym_medical:doctorsim-v0_d238e_00000_0_2022-09-07_13-41-48'
checkpoint_base_url = '/root/ray_results/bert-base-uncased-1/IMPALA_gym_medical:doctorsim-v0_fad65_00000_0_2022-09-16_09-43-12'
checkpoint_base_url = '/root/ray_results/bert-base-uncased-1/IMPALA_gym_medical:doctorsim-v0_328e6_00000_0_2022-09-19_11-50-03'
checkpoint_base_url = '/root/ray_results/bert-base-uncased-1/IMPALA_gym_medical:doctorsim-v0_2c5ba_00000_0_2022-09-19_12-54-18'
checkpoint_base_url = '/root/ray_results/bert-base-uncased-1/IMPALA_gym_medical:doctorsim-v0_5449c_00000_0_2022-09-19_13-31-13'
checkpoint_base_url = '/root/ray_results/bert-base-uncased-1/IMPALA_gym_medical:doctorsim-v0_b0521_00000_0_2022-09-19_15-49-47'
checkpoint_base_url = '/root/ray_results/bert-base-uncased-1/IMPALA_gym_medical:doctorsim-v0_a8108_00000_0_2022-09-19_16-53-59'
checkpoint_base_url = '/pvc/privatefs/data/ray_results/bert-base-uncased-1/IMPALA_gym_medical:doctorsim-v0_85293_00000_0_2022-10-07_09-00-41'
checkpoint_base_url = '/pvc/privatefs/data/ray_results/bert-base-uncased-1/IMPALA_gym_medical:doctorsim-v0_557b3_00000_0_2022-10-31_10-46-54'
checkpoint_base_url = '/pvc/privatefs/data/ray_results/bert-base-uncased-None/IMPALA_gym_medical:doctorsim-v0_1a51d_00000_0_2022-10-31_14-55-47' #Still with [PAD] bug
#checkpoint_base_url = '/pvc/privatefs/data/ray_results/bert-base-uncased-None/IMPALA_gym_medical:doctorsim-v0_108df_00000_0_2022-11-04_15-05-05'
checkpoint_base_url = '/pvc/privatefs/data/ray_results/bert-base-uncased-None/IMPALA_gym_medical:doctorsim-v0_bd510_00000_0_2022-11-11_16-13-12'
checkpoint_base_url = '/pvc/privatefs/data/ray_results/bert-base-uncased-None/IMPALA_gym_medical:doctorsim-v0_7c391_00000_0_2022-11-15_11-41-47'
checkpoint_base_url = '/pvc/privatefs/data/ray_results/bert-base-uncased-10/IMPALA_gym_medical:doctorsim-v0_e2115_00000_0_2022-11-15_15-55-10'

In [7]:
cps = list(map(lambda x: int(x.split('_')[-1]), [e for e in os.listdir(checkpoint_base_url) if e.find('checkpoint')!=-1]))
first, last = min(cps), max(cps)
first, last

FileNotFoundError: [Errno 2] No such file or directory: '/pvc/privatefs/data/ray_results/bert-base-uncased-None/IMPALA_gym_medical:doctorsim-v0_e2115_00000_0_2022-11-15_15-55-10'

In [ ]:
#last = 30000

In [ ]:
#tuner = ray.tune.Tuner.restore('/root/ray_results/bert-base-uncased-None/')

In [ ]:
#vars(tuner._local_tuner)

In [ ]:
with open(checkpoint_base_url + '/params.json' ) as cfg:
    ex_config = json.load(cfg)
ex_config['model']['custom_model_config']['model_path']=cpu_path


In [ ]:
ModelCatalog.register_custom_model('IMPALAFruitfly',IMPALAFruitfly)

config = impala.ImpalaConfig()
config = config.framework('torch').\
            environment(env="gym_medical:doctorsim-v0", env_config = ex_config['env_config']).\
            resources(num_gpus=0, num_gpus_per_worker=0)
config.model = ex_config['model']
#config.callbacks=actionCallbacks
#config.custom_model_config['model_path']=cpu_path
#vars(config)

#

In [ ]:
algorithm = config.build()

In [ ]:
algorithm.restore(checkpoint_base_url + f'/checkpoint_{last:06}')

In [ ]:
env = gym.make("gym_medical:doctorsim-v0", **ex_config['env_config'] )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', use_fast=True)

In [ ]:
W = torch.tensor(algorithm.get_policy().get_weights()['W'])
action_weights = torch.tensor(algorithm.get_policy().get_weights()['action_outputs.weight'])
action_bias_weights = torch.tensor(algorithm.get_policy().get_weights()['action_outputs.bias'])
vocab_size = len(tokenizer)
def synapse_order(prompt, W, vocab_size):
    
    #ids = tokenizer.encode(prompt, add_special_tokens=False)
    ids = prompt
    #print(ids)
    ids = torch.cat( ( torch.tensor(ids), torch.tensor(ids) + vocab_size) ) #KNOWN BUG
    #ids = torch.tensor(ids)
    
    coordinates = torch.stack((torch.zeros_like(ids), ids)).T
    sparse_values = torch.ones_like(ids)*(ids!=0)*(ids!=vocab_size)
    V_A_s = torch.sparse_coo_tensor(coordinates.T, 
                                    #torch.ones_like(ids), 
                                    sparse_values,
                                    (1,2*vocab_size), dtype=torch.float).coalesce()
    V_AxWT = torch.sparse.mm(V_A_s, W.T)
    activation_order = V_AxWT.squeeze().argsort(descending=True)
    return activation_order, V_AxWT

def top_neuron_words(n_neurons, top_n_words, W, activation_order, tokenizer):
    wcs = []
    wcs_dict = []
    vocab = len(tokenizer)
    for NEURON in activation_order[:n_neurons]:
        synapses = W[NEURON][:vocab]
        #synapses = W[NEURON][vocab:] 
        #synapses = torch.nn.functional.softmax(synapses)
        order = synapses.argsort(descending=True) #Important words in the neuron
        neuron = defaultdict(list)
        neuron_synapses = defaultdict(list)
        for i, e in enumerate(order[:top_n_words]):
            neuron_synapses[NEURON.item()].append((e.item(), tokenizer.decode(e.item()), synapses[e].item()))
            neuron[NEURON.item()].append(tokenizer.decode(e.item()))
        wcs_dict.append(neuron_synapses)
        wcs.append(neuron)
    return wcs, wcs_dict

In [ ]:
def hash2str(ff_hash, w, h):
    res = ''
    for i,value in enumerate(ff_hash):
        if i%w == 0:
            res +='\n'
        if value:
            res +='●'
        else:
            res += '○'
            
    return res
h = torch.rand(400)>0.8
print(hash2str(h,50,8))

In [ ]:
#algorithm.compute_single_action(obs)

In [ ]:
action_weights.shape

In [ ]:
eng = ids2English(tokenizer)

In [ ]:
s = env.reset()
#state = [torch.zeros((1, 256)).squeeze(0),torch.zeros((1, 256)).squeeze(0)]
done = False
i = 0
episode = []
while not done:
    #DIA_ATF
    #DIA_LIN
    #if env.get_patient().disease.id != 'DIA_ATF':
    #    s = env.reset()
    #    continue
    #print(f'===========================iteration :{i} Done :[{done}] ===============================')
    #env.render()
    
    #print(f'Current state: {eng(s)}\n')
    a = algorithm.compute_action(s)
    activation_order, V_AxWT = synapse_order(s, W, vocab_size)
    binary_hash = torch.zeros_like(V_AxWT, dtype=torch.bool)
    order = V_AxWT.squeeze().argsort(descending=True)
    trues = order[:50].unsqueeze(0)
    binary_hash = binary_hash.scatter_(dim=1, index = trues, src=torch.ones_like(trues, dtype=torch.bool))
    
    
    # Weight the activation order by the action layer, the hash is just ones
    
    #weighted_activation_order = (V_AxWT.squeeze() * action_weights[a]).argsort(descending=True)
    weighted_activation_order = (binary_hash.squeeze(0) * (action_weights[a] + action_bias_weights[a])).argsort(descending=True)
    #plt.plot(binary_hash.squeeze(0) * (action_weights[a] + action_bias_weights[a]), label = f'step {i}')
    
    tw,wcs_dict = top_neuron_words(6,
                          20,
                          W, 
                          activation_order,
                          #weighted_activation_order,
                          tokenizer)
    for neuron in tw:
        top_words = ''
        for k,v in neuron.items():
            top_words+= f'{k}: ' + ' '.join(v)
        
        #print(top_words)
    #print('--------------------weighted--activations-----')    
    wtw, w_wcs_dict = top_neuron_words(6,
                          20,
                          W, 
                          #activation_order,
                          weighted_activation_order,
                          tokenizer)
    for neuron in wtw:
        top_words = ''
        for k,v in neuron.items():
            top_words+= f'{k}: ' + ' '.join(v)
        
        #print(top_words)

    #print(f'Algorithm action: {a}')
    episode.append((f'Current state: {eng(s)}\n',wcs_dict, binary_hash.squeeze(0) * (action_weights[a] + action_bias_weights[a])))
    #episode.append((f'Current state: {eng(s)}\n',wcs_dict, activation_order))
    s, r, done, info = env.step(a)
    #print(r)
     
    i+=1
    if done and i>10:
        done=False
        s=env.reset()
        episode=[]
        i=0
print(f'===========================iteration :{i} Done :[{done}] ===============================')
env.render()
#plt.legend()

In [ ]:
episode

In [ ]:
from wordcloud import WordCloud
 

for i_episode, (current_state, episode_step, weighted_activation_order) in enumerate(episode[:9]):
    #figure(figsize=(30,30), dpi=600, facecolor='w', edgecolor='k')
    fig, axes = plt.subplots(len(episode_step)//3,3, figsize=(10,10), dpi=100, facecolor='w', edgecolor='k')
    
    
    fig.suptitle(f'Episode step :{i_episode} {current_state}')
    for neuron, ax in zip(episode_step, axes.reshape(-1)):
        cloud_data = defaultdict(float)
        wc = WordCloud(width=256, height=256,background_color='white',max_words=20)
        for k, v in neuron.items():
            for tokenid, word, synapse_weight in v:
                cloud_data[word]=synapse_weight
            #ax.set_title(k)
        wc.generate_from_frequencies(cloud_data)
        ax.imshow(wc, interpolation="bilinear")
        ax.axis("off")
plt.tight_layout(pad=0)    
plt.show()

In [ ]:
from wordcloud import WordCloud
 

for i_episode, (current_state, episode_step, weighted_activation_order) in enumerate(episode[:3]):
    fig, axes = plt.subplots(1,1, figsize=(8,8), dpi=100, facecolor='w', edgecolor='k')
    fig.suptitle(f'Episode step :{i_episode} {current_state}')
    cloud_data = defaultdict(float)
    wc = WordCloud(width=256, height=256,background_color='white',max_words=1000)
    for i_neuron, neuron in enumerate(episode_step):
        
        for k, v in neuron.items():
            
            #print(k, weighted_activation_order[k])
            for tokenid, word, synapse_weight in v:
                cloud_data[word]=(synapse_weight*weighted_activation_order[k]).float().item()
                #cloud_data[word]= synapse_weight
    #ax.set_title(k)
    wc.generate_from_frequencies(cloud_data)
    axes.imshow(wc, interpolation="bilinear")
    axes.axis("off")
plt.tight_layout(pad=0)    
plt.show()

In [ ]:
cloud_data

In [ ]:
plt.imshow(action_weights, cmap='magma')

In [ ]:
vars(env)

In [ ]:
s = env.reset()
done = False
i = 0
env.render()

In [ ]:
def forward(tokenizer, prompt, W, vocab_size, action_weights, bias_weights):
    
    #ids = tokenizer.encode(prompt, add_special_tokens=False)
    ids = prompt
    #print(ids)
    #ids = torch.cat( ( torch.tensor(ids), torch.tensor(ids) + vocab_size) )
    ids = torch.tensor(ids) # THIS IS A BUG, half of the model isn't used
    coordinates = torch.stack((torch.zeros_like(ids), ids)).T
    sparse_values = torch.ones_like(ids)#*(ids!=0)#*(ids!=vocab_size)
    V_A_s = torch.sparse_coo_tensor(coordinates.T, 
                                    #torch.ones_like(ids), 
                                    sparse_values,
                                    (1,2*vocab_size), dtype=torch.float).coalesce()
    V_AxWT = torch.sparse.mm(V_A_s, W.T)
    
    binary_hash = torch.zeros_like(V_AxWT, dtype=torch.bool)
    order = V_AxWT.squeeze().argsort(descending=True)
    trues = order[:50].unsqueeze(0)
    print(trues.shape, binary_hash.shape)
    binary_hash = binary_hash.scatter_(dim=1, index = trues, src=torch.ones_like(trues, dtype=torch.bool))
    logits = torch.matmul(binary_hash.float(), action_weights.T) + bias_weights
    logits = torch.nn.GELU()(logits)
    action = logits.argmax()
    return binary_hash, V_AxWT, logits, action


In [ ]:
binary_hash, V_AxWT, logits, action = forward(tokenizer, torch.tensor(s),W, vocab_size, action_weights, action_bias_weights )
a, states, action_stuff = algorithm.compute_action(s, full_fetch=True)
print(a, action)
plt.plot(logits.squeeze(0))
plt.plot(action_stuff['action_dist_inputs'])

In [ ]:
binary_hash.shape, logits.shape, action.shape

In [ ]:
print(f'Current state: {eng(s)}\n')
a, states, action_stuff = algorithm.compute_action(s, full_fetch=True)

#plt.plot(action[a])
print(f'Action: {a}')
plt.plot(action_stuff['action_dist_inputs'])


In [ ]:
#state = [torch.zeros((1, 256)).squeeze(0),torch.zeros((1, 256)).squeeze(0)]




tw =     top_neuron_words(5,
                          5,
                          W, 
                          synapse_order(tokenizer, s, W, vocab_size)[0], 
                          tokenizer)
for neuron in tw:
    top_words = ''
    for k,v in neuron.items():
        top_words+= f'{k}: ' + ' '.join(v)

    print(top_words)
    #print(f'Neuron {neuron}:{top_words}')

print(f'Algorithm action: {a}')
s, r, done, info = env.step(a)
print(r)
print(f'===========================iteration :{i} Done :[{done}] ===============================') 
i+=1
